# Background Replacement

### This notebook introduces background replacement for input images. This function allows for replacing the background of the image. The implementation allows automatic foregound mask generation using the RMBG-1.4 model. In this notebook, we will provide a step-by-step guide to replacing the background of a given image.

Define an input image

In [ ]:
from PIL import Image
import os, sys

input_path = '../inputs/background-replacement/camera.png'
input_image = Image.open(input_path)

current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(os.path.dirname(current_dir), '..'))
sys.path.append(os.path.join(parent_dir, 'code'))

In [ ]:
from matplotlib import pyplot as plt

plt.imshow(input_image)
plt.axis('off')
plt.show()

Define the number of the inference steps

In [ ]:
steps = 20

Define the prompt for the background replacement

In [ ]:
prompt = 'savanna with wild animals'

Extract foreground image and mask using the RMBG-1.4 model

In [ ]:
from extract_foreground import extract_foreground_image, extract_foreground_mask

forground_image = extract_foreground_image(input_image)
foreground_mask = extract_foreground_mask(forground_image)

In [ ]:
plt.imshow(foreground_mask)
plt.axis('off')
plt.show()

Load the Stable Diffusion model for inpainting

In [ ]:
from diffusers import StableDiffusionInpaintPipeline
import torch

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16,
    )
pipe.to("cuda")

Apply dilation to the mask, to enhance the the results and remove unwanted features, like shadows of the foreground element

In [ ]:
from operations_image import expand_white_areas_outpainting
import numpy as np

size = np.array(input_image).shape[:2]
    
# reverse the mask for outpainting
reversed_mask_array = 255 - np.array(foreground_mask)
reversed_mask_array = Image.fromarray(reversed_mask_array)
reversed_mask_array = expand_white_areas_outpainting(reversed_mask_array, 2) #optional

In [ ]:
from background_replace_sd import background_replace_mask_stablediffusion

output_image = background_replace_mask_stablediffusion(input_image, foreground_mask, prompt, steps)

In [ ]:
plt.imshow(reversed_mask_array)
plt.axis('off')
plt.show()

Infer the model on the input image to obtain the foreground mask

In [ ]:
resized_input_image = input_image.resize((512, 512))
resized_reversed_mask_array = reversed_mask_array.resize((512, 512))
output_image = pipe(prompt=prompt, 
                    image=resized_input_image, 
                    mask_image=resized_reversed_mask_array,  
                    guidance_scale=7.5, 
                    num_inference_steps=steps).images[0]
resized_output_image = output_image.resize((size[1], size[0]))

In [ ]:
plt.imshow(resized_output_image)
plt.axis('off')
plt.show()

Visalise the final result

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

axs[0].imshow(input_image)
axs[0].axis('off')
axs[0].set_title('Original image')

# Display the second image in the right subplot
axs[1].imshow(resized_output_image)
axs[1].axis('off')
axs[1].set_title('Final image')

# Adjust layout to avoid overlap
plt.tight_layout()
plt.show()